![rmotr](https://user-images.githubusercontent.com/7065401/52071918-bda15380-2562-11e9-828c-7f95297e4a82.png)
<hr style="margin-bottom: 40px;">

<img src="https://user-images.githubusercontent.com/7065401/68501079-0695df00-023c-11ea-841f-455dac84a089.jpg"
    style="width:400px; float: right; margin: 0 40px 40px 40px;"></img>

# Reading data from relational databases

In this lesson you will learn how to read SQL queries and relational database tables into `DataFrame` objects using pandas. Also, we'll take a look at different techniques to persist that pandas `DataFrame` objects to database tables.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Hands on! 

In [ ]:
!pip install sqlalchemy

In [ ]:
import pandas as pd

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Read data from SQL database

Reading data from SQL relational databases is fairly simple and pandas support a variety of methods to deal with it.

We'll start with an example using SQLite, as it's a builtin Python package, and we don't need anything extra installed.

In [ ]:
import sqlite3

In order to work with a SQLite database from Python, we first have to connect to it. We can do that using the connect function, which returns a `Connection` object.

We'll use [this example database](http://www.sqlitetutorial.net/sqlite-sample-database/).

In [ ]:
conn = sqlite3.connect('chinook.db')

Once we have a `Connection` object, we can then create a `Cursor` object. Cursors allow us to execute SQL queries against a database:

In [ ]:
cur = conn.cursor()

The `Cursor` created has a method `execute`, which will receive SQL parameters to run against the database.

The code below will fetch the first `5` rows from the `employees` table:

In [ ]:
cur.execute('SELECT * FROM employees LIMIT 5;')

You may have noticed that we didn't assign the result of the above query to a variable. This is because we need to run another command to actually fetch the results.

We can use the `fetchall` method to fetch all of the results of a query:

In [ ]:
results = cur.fetchall()

In [ ]:
results

As you can see, the results are returned as a list of tuples. Each tuple corresponds to a row in the database that we accessed. Dealing with data this way is painful.

We'd need to manually add column headers, and manually parse the data. Luckily, the pandas library has an easier way, which we'll look at in the next section.

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df.head()

Before we move on, it's good practice to close `Connection` objects and `Cursor` objects that are open. This prevents the SQLite database from being locked. When a SQLite database is locked, you may be unable to update the database, and may get errors. We can close the Cursor and the Connection like this:

In [ ]:
cur.close()
conn.close()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Using pandas `read_sql` method

We can use the pandas `read_sql` function to read the results of a SQL query directly into a pandas `DataFrame`. The code below will execute the same query that we just did, but it will return a `DataFrame`. It has several advantages over the query we did above:

- It doesn't require us to create a `Cursor` object or call `fetchall` at the end.
- It automatically reads in the names of the headers from the table.
- It creates a `DataFrame`, so we can quickly explore the data.

In [ ]:
conn = sqlite3.connect('chinook.db')

In [ ]:
df = pd.read_sql('SELECT * FROM employees;', conn)

In [ ]:
df.head()

In [ ]:
df = pd.read_sql('SELECT * FROM employees;', conn,
                 index_col='EmployeeId',
                 parse_dates=['BirthDate', 'HireDate'])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['ReportsTo'].isna().sum()

In [ ]:
df['ReportsTo'].mean()

In [ ]:
df['ReportsTo'] > 1.75

In [ ]:
df['City'] = df['City'].astype('category')

In [ ]:
df.info()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Using pandas `read_sql_query` method

It turns out that the `read_sql` method we saw above is just a wrapper around `read_sql_query` and `read_sql_table`.

We can get the same result using `read_sql_query` method:

In [ ]:
conn = sqlite3.connect('chinook.db')

In [ ]:
df = pd.read_sql_query('SELECT * FROM employees LIMIT 5;', conn)

In [ ]:
df.head()

In [ ]:
df = pd.read_sql_query('SELECT * FROM employees;', conn,
                       index_col='EmployeeId',
                       parse_dates=['BirthDate', 'HireDate'])

In [ ]:
df.head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Using `read_sql_table` method

`read_sql_table` is a useful function, but it works only with [SQLAlchemy](https://www.sqlalchemy.org/), a Python SQL Toolkit and Object Relational Mapper.

This is just a demonstration of its usage where we read the whole `employees` table.

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///chinook.db')

connection = engine.connect()

In [ ]:
df = pd.read_sql_table('employees', con=connection)

In [ ]:
df.head()

In [ ]:
df = pd.read_sql_table('employees', con=connection,
                       index_col='EmployeeId',
                       parse_dates=['BirthDate', 'HireDate'])

In [ ]:
df.head()

In [ ]:
connection.close()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Create tables from `DataFrame` objects

Finally we can persist `DataFrame` objects we've working on in a database using the pandas `to_sql` method.

Although it is easy to implement, it could be a very slow process.

In [ ]:
df.head()

In [ ]:
df.to_sql('employees2', conn)

In [ ]:
pd.read_sql_query('SELECT * FROM employees2;', conn).head()

In [ ]:
#pd.read_sql_query('DROP TABLE employees2;', conn)

## Custom behaviour

The `if_exists` parameter define how to behave if the table already exists and adds a ton of flexibility, letting you decide wheather to `replace` current database data, `append` new data at the end, or simply `fail` if database already exists.

In [ ]:
pd.DataFrame().to_sql('employees2',
                      conn,
                      if_exists='replace')

In [ ]:
pd.read_sql_query('SELECT * FROM employees2;', conn).head()

In [ ]:
df.to_sql('employees2',
          conn,
          if_exists='replace')

In [ ]:
pd.read_sql_query('SELECT * FROM employees2;', conn).head()

In [ ]:
conn.close()

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)